In [1]:
import argparse
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from pytorch_pretrained_bert import BertAdam

from utils.helpers import get_data_loaders
from models import get_model
from utils.logger import create_logger
from utils.utils import *

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="6"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import sys

In [4]:
import torch
import os

In [6]:
poster = torch.load(os.path.join("data_sets/proc/moviescope", 'img_indices2', '0.pt')).long()
poster =poster+4

In [22]:
poster = torch.cat([torch.ones(1).long(),poster,2*torch.ones(1).long()])

In [24]:
poster.size()

torch.Size([258])

In [20]:
torch.ones(1).long()

torch.Size([1])

In [2]:
args = argparse.Namespace(
        bert_model="bert-base-uncased",
        data_path="data_sets/proc/",
        task="moviescope",
        model="mmbt",
        batch_sz=8,
        task_type="multilabel",
        aug=0,
        drop_img_percent=0,
        max_seq_len=512,
        num_image_embeds=3,
        n_workers=8,
        poster="raw"
    )



In [3]:
train_loader, val_loader, test_loader = get_data_loaders(args)

In [4]:
from itertools import cycle

In [17]:
t=zip(train_loader,cycle(val_loader))

In [18]:
len(t)

TypeError: object of type 'zip' has no len()

In [13]:
set_seed(1)
for batch in tqdm(train_loader, total=len(train_loader)):
    txt, segment, mask, img, img_aug, tgt, img_id= batch
    print(img_id)

  0%|          | 2/432 [00:00<02:52,  2.49it/s]

[2643, 1520, 4301, 1656, 3720, 2238, 4719, 4268]
[3218, 3235, 3785, 3323, 2402, 1747, 2403, 825]
[4592, 1695, 2110, 1634, 2288, 3950, 1978, 1273]


  3%|▎         | 15/432 [00:01<00:23, 17.75it/s]

[1467, 5024, 2989, 245, 2283, 1458, 392, 1900]
[1050, 469, 482, 2171, 681, 1580, 3922, 3580]
[492, 2103, 1432, 1104, 1674, 1414, 4595, 805]
[2014, 1234, 4189, 4768, 1041, 4324, 1044, 840]
[4193, 3447, 3497, 1536, 4254, 3346, 2039, 3596]
[3062, 249, 4766, 478, 2168, 4485, 3154, 2779]
[1175, 1410, 3251, 4449, 4853, 3125, 159, 4380]
[1539, 2722, 1376, 151, 4895, 2874, 24, 1387]
[4526, 1007, 712, 4538, 2189, 429, 4978, 1981]
[874, 4057, 4584, 254, 3737, 1564, 1762, 1311]
[4938, 4778, 2849, 129, 2063, 4707, 3131, 3383]
[3830, 2119, 2394, 350, 1977, 3651, 2182, 982]
[1021, 557, 63, 2808, 1601, 653, 3525, 1360]
[2418, 4341, 162, 4561, 148, 3542, 4826, 1428]
[1149, 2035, 2401, 531, 4983, 3745, 714, 4667]
[1595, 2692, 727, 1186, 4031, 253, 1465, 747]
[1526, 3047, 969, 4356, 1972, 2870, 2025, 1764]
[1743, 2197, 2266, 1001, 4500, 4489, 4307, 645]
[2698, 2928, 4655, 3412, 713, 420, 1091, 3425]
[3495, 3809, 849, 3363, 4546, 2048, 2243, 1731]


  9%|▉         | 40/432 [00:01<00:07, 50.58it/s]

...

[4894, 2835, 3658, 1095, 3970, 5, 3340, 3193]
[776, 4045, 205, 239, 15, 3956, 2091, 1694]
[4021, 1514, 3524, 2569, 1524, 155, 2520, 3381]
[4087, 296, 2523, 3499, 2983, 5023, 1123, 2166]
[1702, 4110, 1975, 1939, 1222, 1116, 2802, 3652]
[3155, 4390, 2902, 1732, 3857, 822, 2338, 2165]
[3017, 743, 648, 4674, 1028, 2488, 2495, 2854]
[236, 4198, 909, 2758, 3949, 5010, 2952, 4644]
[700, 248, 572, 1117, 4713, 2812, 1099, 3158]
[312, 4520, 3086, 2292, 4235, 2799, 667, 403]
[1488, 479, 2973, 3397, 4999, 3150, 4820, 3307]
[1359, 827, 1112, 2628, 719, 1728, 1257, 760]
[1495, 2757, 3162, 2761, 503, 622, 4472, 3096]
[4111, 2298, 904, 972, 1513, 2310, 2948, 1768]
[1145, 1838, 1954, 152, 3832, 4438, 1667, 2750]
[2668, 1533, 4747, 3272, 4831, 3711, 508, 3819]
[3019, 4688, 602, 1719, 1494, 3736, 3324, 1631]
[448, 4678, 3740, 3280, 1571, 3598, 4942, 675]
[2097, 3905, 701, 4801, 4926, 3245, 1390, 4513]
[4071, 3520, 4958, 4377, 4353, 4857, 3281, 1550]


 99%|█████████▉| 427/432 [00:09<00:00, 60.53it/s]

[2872, 3994, 62, 2323, 1754, 978, 644, 4443]
[1836, 2260, 125, 4469, 3362, 823, 180, 2485]
[833, 4172, 117, 4403, 4222, 1375, 1898, 3941]
[590, 2936, 1844, 3134, 94, 2818, 2341, 2999]
[1653, 4512, 1748, 2056, 4491, 410, 2961, 4886]
[3121, 49, 1088, 4799, 2085, 515, 1742, 2409]
[1822, 3221, 4488, 3450, 4066, 4180, 3664, 2890]


100%|██████████| 432/432 [00:10<00:00, 42.17it/s]

[4897, 4980, 2963, 1141, 1577, 4475, 1202, 51]
[643, 494, 419, 4647, 3714, 2937, 3952, 3828]
[366, 1254, 4144, 1891, 3645, 4414, 183, 3527]
[4870, 4361, 4589, 1562, 850, 1172, 2607, 352]
[1745]


In [23]:
set_seed(1)
for batch in tqdm(zip(train_loader,train_loader), total=len(train_loader)):
    txt, segment, mask, img, img_aug, tgt, img_id= batch[1]
    txt2, segment2, mask2, img2, img_aug2, tgt2, img_id2= batch[0]
    print(np.array(img_id)-np.array(img_id2))

  3%|▎         | 14/432 [00:00<00:06, 63.75it/s]

[  553 -1072 -3658  2116   724   328 -2173  -784]
[  849  1395 -3090  -572   712 -1208 -1259  3592]
[-3480  -606  -988  3244 -2034 -2104  -633   316]
[-1346 -2604 -1333  3321  1430  3035   629 -1509]
[ 3018  3136  3255 -1803   598  1603 -1769 -1748]
[ 4135  2037  1929  -167  3233  2986 -4169  2907]
[ -928  1626 -2126 -4454  -856 -3083  -993  -334]
[-1294 -3003   106  -510 -3518 -2659   628 -3063]
[-2896  2582  -415  2336 -1175 -1862  1292 -2746]
[ 3059  1596  1115 -1175 -2902   971  1518   -15]
[1122 1417   -8 3907 -923  406 2161 -570]
[  485  -144  3447 -2319  2189  2718  -281 -1732]
[-728  341 -778 3248  982 1593 1132 1445]
[-4283 -3170   137  1472  1770 -1773   766 -1390]
[  704  -274   286  1323 -1544  -775  2210  3424]
[-407 -277 3529  969 2512 3795  852 2256]
[  759  -787  1138 -4486   888  1039 -4344  1600]
[ 1221  1879  1442   -67  -386 -2248  -657 -4310]
[-1091 -2026  2217   -78   966  4575 -1329  3415]
[ 2263    -4  2284 -1250  2486  1111  1172  2715]
[  215   180   798  -380

 10%|▉         | 42/432 [00:00<00:04, 97.48it/s]

[-4440   307  1275  1865   799 -1082   671 -2989]
[   92 -1313  4001 -1134 -2366  2315  1393 -1072]
[ 3907  2925 -1060   497  4509 -1090  3094  2227]
[  327 -2388   659  -685  2524  3653  -581  1063]
[-1647   686  -540  2305  1152 -3272  3400  2669]
[ 3395   668 -1955 -1127  2978 -3733  2269   321]
[ 4058  2005   557 -1611 -1508 -4724 -3540 -1105]
[ 3756   355 -1389  -183  1626  -579   798  2521]
[-3042  -325  1603  4759  1698  -190 -2206  4131]
[-1505  2106   673  3643 -2285  2853  -586  2811]
[   63  -540 -2771   294   696 -1083 -2751 -3071]
[-1990  2831   -99 -3256    76 -1197  -856   470]
[  240  -764 -3096 -1032  3319  1154 -2423 -3687]
[  625  4757 -3290   830   224   390 -2373  -203]
[-2383 -2815  -475  1915  -141 -2220 -2185 -1152]
[-1497  -633 -3170  1763   403 -3834  1402  3393]
[-2035 -3251  1186    92  -630  -260 -1731 -1089]
[ -191  2459  -594  -435 -1170  2063  1419 -3793]
[ -843  2213 -3636  2569  -188  2442  -777  2462]
[ 1588 -1840  2006  4043  1395 -3683 -3322    10]


 16%|█▌        | 70/432 [00:00<00:03, 111.48it/s]

...

[  390  1662 -1350  -693 -2175 -1002   474   741]
[ 1231 -1553   986  2374  2336  3634   408  2539]
[-3741  2595 -3636  3885  1990   835  1703  -358]
[  -31  -276 -1733 -2007 -1801  -560 -2716   450]
[  388 -1996   407   350 -1426  1604 -3249 -2191]
[-1822  -435   168 -2514 -1570 -2643  -538  -927]
[-1317  -743   699  1489  1331   562  -492  2935]
[  144   374  3961  1052    69 -3671   -51  2961]
[   40  2965  -938  -752   783  3154 -2176 -2063]
[ 1054   474  1117 -3324  1253 -1812  1513  -525]
[ 3872  1258  3710  1739  1655 -1873   249   243]
[  759   498  2389  -139  3963  1773 -2549  1211]
[ -233 -2818 -1771 -2222  1747  1861    36  -956]
[ 4014 -3395   -21 -4242    86 -4152  -327  1660]
[-3030 -1076   702  -940 -3491  1855 -2432  1096]
[ -731 -1228  2348  -681 -1074 -1392  3524   226]
[ 4253  -781  1983  1526  -998   216   138 -1249]
[-1578 -2674 -2102  3337   573  1489  1416  2729]
[-1165  3865   342   189 -1280 -3590  -394 -2642]
[-1522 -2162  1189 -2164  -877   943  3340  -591]


 97%|█████████▋| 417/432 [00:03<00:00, 109.53it/s]

[-1399  3010  -943  4767  1174  4038 -2362 -2758]
[-1114   143   978 -1129   550  1985  2798   650]
[-3895  1831   576  -405 -3417  3127  -380 -2067]
[ 2454  1260  2932  4668   120   183 -1946  -203]
[ 1055 -3692 -1232   141  -882  3576 -3002 -1491]
[-1126  3168  1636  1130  2952  1796  2257 -1441]
[ -773 -2030   403  1575 -1737  3687 -1292 -1198]
[3439  714 2794 4527 2805  -30 1778 -198]
[ -756 -1292 -1360   771  1901  2552   530  -391]
[ -407  1091   381 -1558   905 -1676   470  2019]
[ 2668   617  -229  2853  3183  1401 -2345   515]
[-2507 -2552   206 -1075 -1285  -541  -198 -1056]
[-1987  1165  3248  -654  3610 -1078   278   596]
[ 2612  -498  3495  -579 -1894 -4861   127 -3171]
[ 1277  2063   804  2279  -991 -1793  -917 -1532]
[ 3824 -4379  -560  -163   538 -1700   302   971]
[ -679  4417    80 -2646 -4135   267  -896 -3222]
[  141  3226  3356 -1199  3858  1142  2303  3485]
[ 2880 -2232  -962  1873   952  1431 -2982 -2344]
[-3536 -1224  3391  -240  2609  1627 -2167  2847]
[ 2584  

100%|██████████| 432/432 [00:04<00:00, 104.79it/s]

[  279 -3491 -2236  -997  1208 -3487 -3491  3649]
[-1254 -3001  3129 -3660   -85 -2049 -1014 -1540]
[  522   148 -2878 -4224   310 -1985 -1188  1066]
[ 106  353 3884 1541 1707 1951  300  145]
[ 1458 -2214  2309 -4338 -3093  -107  4273  1546]
[ 3902 -3972   172 -2733 -1865   291  1653   141]
[2264 1201 2701  888 1784 1196 1541  707]
[-1437 -2711 -1571  2414 -2202  1738 -2713 -2573]
[ -278   504  3826 -3030  -510  2617  1389    79]
[ 2957 -2680   -27  1438 -2312  -634   768 -2582]
[-3380 -2694 -2164  -371  3345 -3053   217  3043]
[ 3102   146  3920 -3700 -1321  1017 -1061 -1355]
[ 2459  1254 -2554  1168  -422 -3159  3595 -1725]
[-3779 -3283 -3212  -916  2705  3506  -326  3010]
[-91]


In [20]:
import numpy as np

In [42]:
for i,  in enumerate(zip(train_loader,cycle(val_loader))):
    txt, segment, mask, img, img_aug, tgt, img_id= batch[0]
    txt, segment, mask, img, img_aug, tgt, img_id= batch[0]
    print(img_id)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [2]:
from datasets import load_dataset

In [3]:
dataset = load_dataset('oscar', 'unshuffled_deduplicated_it')

Reusing dataset oscar (/002/usuarios/ivonne.monter/.cache/huggingface/datasets/oscar/unshuffled_deduplicated_it/1.0.0/84838bd49d2295f62008383b05620571535451d84545037bb94d6f3501651df2)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 28522082
    })
})

In [ ]:
from tqdm.auto import tqdm

text_data = []
file_count = 0

for sample in tqdm(dataset['train']):
    sample = sample['text'].replace('\n', '')
    text_data.append(sample)
    if len(text_data) == 10_000:
        # once we git the 10K mark, save to file
        with open(f'dataset/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# after saving in 10K chunks, we will have ~2082 leftover samples, we save those now too
with open(f'dataset/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))

  0%|          | 0/28522082 [00:00<?, ?it/s]